In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import py4j

spark = SparkSession.\
        builder.\
        appName("pyspark-notebook-test-postgres-results").\
        master("spark://spark:7077").\
        config('spark.jars.packages', 'org.postgresql:postgresql:42.5.1').\
        getOrCreate()

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a5631c3b-ecf6-4db8-90f6-9b81823614fb;1.0
	confs: [default]
	found org.postgresql#postgresql;42.5.1 in central
	found org.checkerframework#checker-qual;3.5.0 in central
:: resolution report :: resolve 103ms :: artifacts dl 5ms
	:: modules in use:
	org.checkerframework#checker-qual;3.5.0 from central in [default]
	org.postgresql#postgresql;42.5.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------------------------------------

23/01/24 14:59:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
gen_connection_string = "jdbc:postgresql://test-postgres-db-%d:5432/db"
dbtable = "tbl"
username = "auser"
passwd = "1234"

In [3]:
class TestPostgresDataInSync:
    def __init__(self, spark_session, gen_connection_string, dbtable, username, passwd):
            self.df1 = self.load(spark_session, gen_connection_string%(1), dbtable, username, passwd).cache()
            self.df2 = self.load(spark_session, gen_connection_string%(2), dbtable, username, passwd).cache()
            
            
    def load(self, spark_session, connection_string, dbtable, username, passwd):
        return spark_session \
                        .read \
                        .format("jdbc") \
                        .option("url", connection_string) \
                        .option("driver", "org.postgresql.Driver") \
                        .option("dbtable", dbtable) \
                        .option("user", username) \
                        .option("password", passwd) \
                        .load()
    
    def count(self):
        cnt1 = self.__count(self.df1)
        cnt2 = self.__count(self.df2)
        print("Data available in test-postgres-db-1 : %d"%(cnt1))
        print("Data available in test-postgres-db-2 : %d"%(cnt2))
    
    def __count(self, df):
        return df.count()
    
    def count_distinct(self):
        cnt1 = self.__count_distinct(self.df1)
        cnt2 = self.__count_distinct(self.df2)
        print("Distinct data available in test-postgres-db-1 : %d"%(cnt1))
        print("Distinct data available in test-postgres-db-1 : %d"%(cnt2))
        assert cnt1 == cnt2
    
    def __count_distinct(self, df):
        return df.select(col("val1"), col("val2")).distinct().count()
    

    def assert_dataframe_equality(self):
        assert self.__count_distinct(self.df1) == self.__count_distinct(self.df2) == \
                    self.__count_distinct( 
                                self.df1.join(self.df2, \
                                            ["val1", "val2"], \
                                              "inner")
                    )

In [4]:
tester = TestPostgresDataInSync(spark, gen_connection_string, dbtable, username, passwd)

In [5]:
tester.count()

Data available in test-postgres-db-1 : 524
Data available in test-postgres-db-2 : 262


In [6]:
tester.count_distinct()

Distinct data available in test-postgres-db-1 : 259
Distinct data available in test-postgres-db-1 : 259


In [7]:
tester.assert_dataframe_equality()